**Chapter 04: Use of OntoGPT**

Ray LeClair \<2024-07-08 Mon\>

# Objectives

Since the NCBI Cell pipeline calls for the use of a Large Language
Model (LLM) to process publications associated with CELLxGENE
datasets, the objective of this document is to:

-   Investigate the use of OntoGPT for publication processing

OntoGP is a Python package for extracting structured information from
text using LLMs, instruction prompts, and ontology-based grounding.

See: [OntoGPT](https://monarch-initiative.github.io/ontogpt/)

## Background

The primary extraction method currently implemented in OntoGPT is
SPIRES: Structured Prompt Interrogation and Recursive Extraction of
Semantics, which:

-   Implements a Zero-Shot Learning (ZSL) approach to extracting nested
    semantic structures from text

-   Uses OpenAI GPT models through their API

This demonstration use the `pubmed-annotate` function to retrieve full
text of a publicaiton given a PubMed identifier (PMID), then performs
extraction on the text with SPIRES using the "Cell Type" template.

[- SPIRES: Structured Prompt Interrogation and Recursive Extraction of Semantics](https://academic.oup.com/bioinformatics/article/40/3/btae104/7612230)

-   [pubmed-annotate](https://monarch-initiative.github.io/ontogpt/functions/#pubmed-annotate)
-   [Cell Type](https://monarch-initiative.github.io/ontogpt/cell_type/)

## Development environment

See: [Introduction.ipynb](Introduction.ipynb)

### Jupyter Notebook

Launch Jupyter Notebook from a terminal in which `.zshenv` has been
sourced, and the virtual environment has been activated.

### Emacs Org Mode

Launch Emacs from a terminal in which `.zshenv` has been sourced, then
evaluate this code block to activate the virtual environment:

``` commonlisp
(pyvenv-activate "../../.venv")
```

# Investigate the use of OntoGPT for publication processing

To begin, we import modules, and assign module scope variables:

In [ ]:
import os
import subprocess
from traceback import print_exc

DATA_DIR = "../data"

ONTOGPT_DIR = f"{DATA_DIR}/ontogpt"


Then recall that in [Chapter-02-E-Utilities.org](Chapter-02-E-Utilities.org) we saw how to get the
PMID for a title:

In [ ]:
from CELLxGENE import get_lung_obs_and_datasets, get_title
from E_Utilities import get_pmid_for_title
try:
    lung_obs, lung_datasets = get_lung_obs_and_datasets()
    citation = lung_datasets["citation"].iloc[0]
    title = get_title(citation)
    pmid = get_pmid_for_title(title)
except Exception:
    print_exc()
print(f"PMID: {pmid} found for title: {title}")


Since the `pubmed-annotate` function of OntoGPT is run on the command
line, we use Python's `subprocess` module, caching results in a file
to prevent duplicate processing:

In [ ]:
def run_ontogpt_pubmed_annotate(pmid):
    """Run the OntoGPT pubmed-annotate function for the specified PMID
    associated with a dataset.

    Parameters
    ----------
    pmid : str
       The PubMed identifier found

    Returns
    -------
    None
    """
    # Run OntoGPT pubmed-annotate function, if needed
    if pmid is None:
        return
    output_filename = f"{pmid}.out"
    output_filepath = f"{ONTOGPT_DIR}/{output_filename}"
    if not os.path.exists(output_filepath):
        print(f"Running ontogpt pubmed-annotate for PMID: {pmid}")
        subprocess.run(
            [
                "ontogpt",
                "pubmed-annotate",
                "--template",
                "cell_type",
                pmid,
                "--limit",
                "1",
                "--output",
                output_filepath,
            ],
        )
        print(f"Completed ontogpt pubmed-annotate for PMID: {pmid}")

    else:
        print(f"Ontogpt pubmed-annotate output for PMID: {pmid} exists")


Now call the function with the PMID obtained earler:

In [ ]:
try:
    run_ontogpt_pubmed_annotate(pmid)
except Exception:
    print_exc()


Next, in Chapter 05 we'll use the results produced by NS-Forest to
populate an ArangoDB database graph.

See:

-   [Chapter-05-ArangoDB.ipynb](Chapter-05-ArangoDB.ipynb)
-   [Chapter-03-NS-Forest.org](Chapter-03-NS-Forest.org)